To install and import all the Needed packages

In [ ]:
!pip install tensorflow keras keras-tuner numpy wget
import wget h5py keras_tuner os
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Reshape,
    Conv2D,
    Dropout,
    Flatten,
    BatchNormalization,
)
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
)


First choose the dataset that will be used on the next steps

Datasets available:
1. 64x16 Single Pilot 80_Ntrain -5_SNR: 
2. 8x4 Single Pilot 80_Ntrain -5_SNR: 
3. 8x4 Single Pilot 80_Ntrain 0_SNR: 

To download using the notebook change CHOSEN_DATASET on the next script to the desired dataset number(Example: "CHOSEN_DATASET = 1" to get the 64x16 Single Pilot 80_Ntrain -5_SNR dataset).

In [ ]:
CHOSEN_DATASET = 2
BASE_LINK="https://nextcloud.lasseufpa.org/remote.php/webdav/Lasse/Projects/UFA27_SmartHaul/MIMO-Hybrid-Channel-Estimation/Datasets/"
LINKS={
    1:"Pilots_SNR_(-5dB)_64x16(80pilots)_single_sub.hdf5",
    2:"Pilots_SNR_(-5dB)_8x4(80pilots)_single_sub.hdf5",
    3:"Pilots_SNR_(0dB)_8x4(80pilots)_single_sub.hdf5",
    "channels_64x16_single_sub":"Channels_64x16_single_sub.hdf5",
    "channels_8x4_single_sub":"Channels_8x4_single_sub.hdf5",
}
if not os.path.isfile(LINKS[CHOSEN_DATASET]):
    wget.download(BASE_LINK + LINKS[CHOSEN_DATASET])
if CHOSEN_DATASET > 1:
    if not os.path.isfile(LINKS["Channels_8x4_single_sub"]):
        wget.download(BASE_LINK + LINKS["Channels_8x4_single_sub"])
        CHOSEN_CHANNELS = "Channels_64x16_single_sub.hdf5"
elif not os.path.isfile(LINKS["channels_64x16_single_sub"]):
    wget.download(BASE_LINK + LINKS["Channels_64x16_single_sub"])
    CHOSEN_CHANNELS = "Channels_8x4_single_sub.hdf5"

Change the following values according to the dataset

In [ ]:

# Keras tuner parameters
MAX_TRIALS = 200
BATCH_SIZE = 32
EPOCHS = 1000


# Change these values acording to the dataset
NR = 8 
NT = 4 
LR = 4 
CHANNELS_USED = 9000
SUBCARIERS_USED_TRAIN = 1
SUBCARIERS_USED_OUT = 1
NC = 10000
NTRAIN = 80
CHANNELS_USED_TEST = NC - CHANNELS_USED



# Directory of the Channels Matrix file
CHANNELS_FILE = h5py.File(LINKS[CHOSEN_DATASET], "r")


# Directory of the Pilots Matrix file
PILOTS_FILE = h5py.File(CHOSEN_CHANNELS, "r")


input_shape = (NTRAIN * LR, SUBCARIERS_USED_TRAIN)
output_shape = (NR, NT, SUBCARIERS_USED_OUT)

This function will be used as a metric in keras to calculate the NMSE(Normalized Mean Squared Error) for each train or validation example that will be iterated in the training process, providing clear metrics to the whole process and enabling the use of early_stopping.

In [ ]:
# Declaring the function to calculate NMSE for each batch
def NMSEtrainComplex(y_true, y_pred):
    sub = y_pred[:, :, :] - y_true[:, :, :]
    H_k = y_true[:, :, :]
    nmse = tf.norm(sub, ord="fro", axis=(1, 2)) ** 2
    den = tf.norm(H_k, ord="fro", axis=(1, 2)) ** 2

    result = (nmse / den)

    return 10*tf.experimental.numpy.log10(result)

Now the NN model will be crated, a chain of Dense layers is being used with relu activation.

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(BatchNormalization())
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
model.add(Dense((512), activation="relu"))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(Dense(np.prod(output_shape), activation="linear"))
model.add(Reshape(output_shape))
model.compile(
    loss="mean_squared_error", optimizer="adam", metrics=[NMSEtrainComplex]
    )

Train the Model

In [ ]:

training_inputData = PILOTS_FILE["pilots"][:NC, :, :SUBCARIERS_USED_TRAIN]
training_outputData = CHANNELS_FILE["channels"][:NC, :, :, :SUBCARIERS_USED_OUT]


history = model.fit(
        training_inputData,
        training_outputData,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        verbose=1,
        shuffle=True,
        validation_split=0.2,
        callbacks=[
            EarlyStopping(
                monitor="val_NMSEtrainComplex",
                min_delta=5e-3,
                patience=40,
                restore_best_weights=True,
            ),
            ModelCheckpoint(
                filepath="./model",
                monitor="val_NMSEtrainComplex",
                verbose=1,
                save_best_only=True,
            ),
            ReduceLROnPlateau(
                factor=0.5,
                min_delta=5e-2,
                patience=20,
                cooldown=5,
                verbose=1,
                min_lr=1e-7,
            ),
        ],
    )
